In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# === Параметры ===
WINDOW_SIZE = 512      # длина окна (сэмпла)
STRIDE = 256            # шаг окна
BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 1e-3
CONFIDENCE_THRESHOLD = 0.7  # порог уверенности для "отказа от ответа"

In [3]:
def load_csv_files(folder, label):
    samples = []
    labels = []
    for fname in os.listdir(folder):
        if fname.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder, fname))
            v = df['vibration'].values
            # Разбиваем на окна
            for start in range(0, len(v) - WINDOW_SIZE + 1, STRIDE):
                window = v[start:start+WINDOW_SIZE]
                samples.append(window)
                labels.append(label)
    return samples, labels

In [4]:
# === Класс датасета ===
class VibrationDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)  # [N, 1, WINDOW_SIZE]
        self.labels = torch.tensor(labels, dtype=torch.float32)           # [N]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [5]:
# === Модель ===
class Simple1DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, 5, padding=2)
        self.pool2 = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(32 * (WINDOW_SIZE // 4), 64)
        self.fc2 = nn.Linear(64, 1)
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x).squeeze(1)

In [6]:
# Укажите пути к папкам с .csv файлами
normal_folder = 'csv_data/normal'  # без дефекта
defect_folder = 'csv_data/defect'  # с дефектом

print('Загрузка данных...')
X_normal, y_normal = load_csv_files(normal_folder, 0)
X_defect, y_defect = load_csv_files(defect_folder, 1)

X = np.array(X_normal + X_defect)
y = np.array(y_normal + y_defect)

print(f'Всего сэмплов: {len(X)}')

# Разделяем на train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

train_dataset = VibrationDataset(X_train, y_train)
val_dataset = VibrationDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Модель, оптимизатор, функция потерь
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Simple1DCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCELoss()

Загрузка данных...
Всего сэмплов: 12319


In [7]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    avg_loss = total_loss / len(train_dataset)

    # Валидация
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item() * xb.size(0)
            predicted = (preds > 0.5).float()
            correct += (predicted == yb).sum().item()
            total += xb.size(0)
    val_loss /= len(val_dataset)
    val_acc = correct / total
    print(f'Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

Epoch 1/3 | Train Loss: 0.0771 | Val Loss: 0.0012 | Val Acc: 0.9996
Epoch 2/3 | Train Loss: 0.0006 | Val Loss: 0.0003 | Val Acc: 1.0000
Epoch 3/3 | Train Loss: 0.0001 | Val Loss: 0.0002 | Val Acc: 1.0000


In [8]:
def predict_with_rejection(model, sample, threshold=CONFIDENCE_THRESHOLD):
        model.eval()
        with torch.no_grad():
            x = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
            prob = model(x).item()
            if abs(prob - 0.5) < (threshold - 0.5):
                return 'Неуверен'
            return 'Дефект' if prob > 0.5 else 'Нет дефекта'

In [9]:
torch.save(model.state_dict(), 'vibration_cnn.pth')


In [10]:

def load_csv_file_to_windows(file_path, window_size=WINDOW_SIZE, stride=STRIDE):
    df = pd.read_csv(file_path)
    v = df['vibration'].values
    windows = []
    for start in range(0, len(v) - window_size + 1, stride):
        window = v[start:start+window_size]
        windows.append(window)
    return windows


In [11]:
samples = load_csv_file_to_windows("csv_data/normal/Normal_0.csv")
result = predict_with_rejection(model, v, threshold=CONFIDENCE_THRESHOLD)
print(result)

NameError: name 'v' is not defined

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

# ==== Модель ====
class Simple1DCNN(nn.Module):
    def __init__(self, window_size=512):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, 5, padding=2)
        self.pool2 = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(32 * (window_size // 4), 64)
        self.fc2 = nn.Linear(64, 1)
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x).squeeze(1)

# ==== Функция для разбиения на окна ====
def load_csv_file_to_windows(file_path, window_size=512, stride=256):
    df = pd.read_csv(file_path)
    v = df['vibration'].values
    windows = []
    for start in range(0, len(v) - window_size + 1, stride):
        window = v[start:start+window_size]
        windows.append(window)
    return windows

# ==== Основной процесс ====
def predict_file(file_path, model_path='vibration_cnn.pth', window_size=512, stride=256, threshold=0.5, confidence_band=0.2):
    # Загрузка модели
    model = Simple1DCNN(window_size=window_size)
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.eval()

    # Загрузка и разбиение файла
    windows = load_csv_file_to_windows(file_path, window_size=window_size, stride=stride)
    if not windows:
        print("В файле недостаточно данных для одного окна.")
        return

    windows_tensor = torch.tensor(windows, dtype=torch.float32).unsqueeze(1)  # [N, 1, window_size]

    # Предсказания
    with torch.no_grad():
        outputs = model(windows_tensor)
        probs = outputs.numpy()

    # Анализ результатов
    verdicts = []
    for i, prob in enumerate(probs):
        if abs(prob - 0.5) < confidence_band:
            verdict = f'Окно {i}: Неуверен (prob={prob:.3f})'
            verdicts.append('Неуверен')
        elif prob > threshold:
            verdict = f'Окно {i}: Дефект (prob={prob:.3f})'
            verdicts.append('Дефект')
        else:
            verdict = f'Окно {i}: Нет дефекта (prob={prob:.3f})'
            verdicts.append('Нет дефекта')
        print(verdict)

    # Итоговое решение для файла
    if 'Дефект' in verdicts:
        print('\nВ файле обнаружен дефект!')
    elif all(v == 'Неуверен' for v in verdicts):
        print('\nМодель не уверена ни в одном окне.')
    else:
        print('\nДефект не обнаружен.')




predict_file("csv_self_data/defect/50Hz_accelerometer.csv")


Окно 0: Дефект (prob=1.000)
Окно 1: Дефект (prob=1.000)
Окно 2: Нет дефекта (prob=0.002)
Окно 3: Нет дефекта (prob=0.000)
Окно 4: Нет дефекта (prob=0.000)
Окно 5: Дефект (prob=1.000)
Окно 6: Дефект (prob=1.000)
Окно 7: Нет дефекта (prob=0.015)
Окно 8: Нет дефекта (prob=0.000)
Окно 9: Нет дефекта (prob=0.000)
Окно 10: Дефект (prob=1.000)
Окно 11: Дефект (prob=1.000)
Окно 12: Нет дефекта (prob=0.096)
Окно 13: Нет дефекта (prob=0.000)
Окно 14: Нет дефекта (prob=0.000)
Окно 15: Дефект (prob=1.000)
Окно 16: Дефект (prob=1.000)
Окно 17: Нет дефекта (prob=0.006)
Окно 18: Нет дефекта (prob=0.002)
Окно 19: Нет дефекта (prob=0.001)
Окно 20: Дефект (prob=1.000)
Окно 21: Дефект (prob=1.000)
Окно 22: Нет дефекта (prob=0.010)
Окно 23: Нет дефекта (prob=0.000)
Окно 24: Нет дефекта (prob=0.000)
Окно 25: Дефект (prob=1.000)
Окно 26: Дефект (prob=1.000)
Окно 27: Нет дефекта (prob=0.001)
Окно 28: Нет дефекта (prob=0.000)
Окно 29: Нет дефекта (prob=0.000)
Окно 30: Дефект (prob=1.000)
Окно 31: Дефект (prob

/tmp/ipykernel_18153/937123188.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location='cpu'))
